### 레시피 크롤링?
- 데이터셋: 만개의 레시피 or 우리의 식탁 = "https://wtable.co.kr/recipes"
- 카테고리: 어떻게 나눌 것인가?
- 가져올 데이터: 요리명, 링크, 재료
- 인식을 위한 id 필요?


In [54]:
import os
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import collections
from requests.exceptions import ConnectionError
import re

In [2]:
# soup error 방지
if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21228\1574705726.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if not hasattr(collections, 'Callable'):


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
# User Agent 정보 추가
agent_option = webdriver.ChromeOptions()
user_agent_string = 'Mozilla/5.0'
agent_option.add_argument('user-agent'+user_agent_string)

In [14]:
def wait_for_page_load(driver):
    WebDriverWait(driver, 10).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )

In [18]:
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 스크롤을 맨 아래로 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # 페이지 로딩 대기
        time.sleep(2)  # 로딩 대기 시간 조정 가능

        # 새로운 높이를 가져오기
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # 더 이상 스크롤 할 수 없다면 종료
        if new_height == last_height:
            break
        
        last_height = new_height

In [103]:
driver = webdriver.Chrome(options=agent_option)
driver.get('https://wtable.co.kr/recipes')
wait_for_page_load(driver)

# <li class="RecipeListstyle__Category-sc-1s9b4ly-17 erDeOn">한식</li>

try:
    # 요소가 클릭 가능할 때까지 기다림
    # element = WebDriverWait(driver, 10).until(
    #     # EC.element_to_be_clickable((By.CSS_SELECTOR, '.RecipeListstyle__Category-sc-1s9b4ly-17.erDeOn'))
    #     EC.element_to_be_clickable((By.XPATH, "//li[text()='한식']"))
    # )
    
    # ul 안의 모든 li 요소 찾기
    ul_element = driver.find_element(By.CLASS_NAME, 'RecipeListstyle__Categories-sc-1s9b4ly-16')  # ul 클래스 이름으로 찾기
    li_elements = ul_element.find_elements(By.TAG_NAME, 'li')  # 모든 li 태그 찾기

    # "한식"이라는 텍스트가 있는 li 찾기
    for li in li_elements:
        if li.text == "한식":
            print("한식 항목을 찾았습니다!")
            li.click()  # 한식으로 이동하고 싶다면 클릭
            break
    
    # element.click()  # 클릭
    
    # 페이지 스크롤을 맨 아래로 내리기
    scroll_to_bottom(driver)

    # 현재 페이지의 HTML 코드 저장
    page_source = driver.page_source

    # HTML 코드를 파일로 저장
    with open('page_source_ko_2.html', 'w', encoding='utf-8') as f:
        f.write(page_source)

    print("HTML 코드가 'page_source_ko.html'로 저장되었습니다.")

except Exception as e:
    print(f"Error: {e}")
finally:
    driver.quit()


한식 항목을 찾았습니다!
Error: 'NoneType' object is not callable


In [104]:
# 드라이버 초기화
driver = webdriver.Chrome(options=agent_option)
driver.get('https://wtable.co.kr/recipes')
wait_for_page_load(driver)  # 페이지 로드 대기 함수가 있다고 가정

try:
    # ul 안의 모든 li 요소 찾기
    ul_element = driver.find_element(By.CLASS_NAME, 'RecipeListstyle__Categories-sc-1s9b4ly-16')  # ul 클래스 이름으로 찾기
    li_elements = ul_element.find_elements(By.TAG_NAME, 'li')  # 모든 li 태그 찾기

    # "한식"이라는 텍스트가 있는 li 찾기
    for li in li_elements:
        if li.text == "한식":
            print("한식 항목을 찾았습니다!")
            li.click()  # 한식 항목 클릭
            break
    
    # 페이지 스크롤을 맨 아래로 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 스크롤 함수 대체

    # 현재 페이지의 HTML 코드 저장
    page_source = driver.page_source

    # HTML 코드를 파일로 저장
    with open('page_source_ko_2.html', 'w', encoding='utf-8') as f:
        f.write(page_source)

    print("HTML 코드가 'page_source_ko_2.html'로 저장되었습니다.")

except Exception as e:
    print(f"Error: {e}")
finally:
    driver.quit()

한식 항목을 찾았습니다!
HTML 코드가 'page_source_ko_2.html'로 저장되었습니다.


In [120]:
url = open('./wtable.co.kr_recipes.html', encoding='utf-8')
# wtable.co.kr_recipes.html
# page_source_ko.html
# html = requests.get(url)
soup = BeautifulSoup(url, 'html.parser')
url.close()

recipes = {'title':[],
           'subtitle':[],
           'img_url':[],
           'level':[],
           'link':[],
           'time':[],
           'element':[]}

# <div class="RecipeItemstyle__Layout-sc-1gt7vz8-0 erZvWP">
# <img src="https://static.wtable.co.kr/image/production/service/recipe/2344/9a1bd1e4-cbf9-4e2c-857a-56c95078bf4c.jpg?size=500x500" class="RecipeItemstyle__ImageStyle-sc-1gt7vz8-1 duQJWI">
# <p class="RecipeItemstyle__SubTitle-sc-1gt7vz8-2 LxJcT">LG 광파오븐 레시피</p>
# <p class="RecipeItemstyle__Title-sc-1gt7vz8-3 hpYiJK">고추전</p>
# <div class="RecipeItemstyle__Summary-sc-1gt7vz8-4 XtlHh">
# <div class="RecipeItemstyle__Time-sc-1gt7vz8-6 RecipeItemstyle__Level-sc-1gt7vz8-7 gvzBms">
# <svg xmlns="http://www.w3.org/2000/svg" width="12" height="12" fill="none" viewBox="0 0 12 12">
# <path stroke="#757575" d="M10.1.996v10m-4-7.992v8m-4-6v6"></path></svg><span class="RecipeItemstyle__SummaryText-sc-1gt7vz8-5 MiXMV">중급</span></div>
# <div class="RecipeItemstyle__Time-sc-1gt7vz8-6 kKXboO"><svg xmlns="http://www.w3.org/2000/svg" width="12" height="12" fill="none" viewBox="0 0 12 12"><circle cx="6" cy="6" r="5.5" stroke="#757575"></circle>
# <path stroke="#757575" d="M5.5 2.5v4h4"></path></svg><span class="RecipeItemstyle__SummaryText-sc-1gt7vz8-5 MiXMV">30분</span></div></div></div>

div_list = soup.find_all('div', {'class':'RecipeItemstyle__Layout-sc-1gt7vz8-0 erZvWP'})

for div in div_list:
    # 제목과 서브타이틀 찾기
    title = div.find('p', {'class': 'RecipeItemstyle__Title-sc-1gt7vz8-3 hpYiJK'})
    subtitle = div.find('p', {'class': 'RecipeItemstyle__SubTitle-sc-1gt7vz8-2 LxJcT'})
    
    # 이미지 URL 찾기
    img_url = div.find('img', {'class': 'RecipeItemstyle__ImageStyle-sc-1gt7vz8-1 duQJWI'})['src']
    
    # 난이도와 소요 시간 찾기
    level = div.find('span', {'class': 'RecipeItemstyle__SummaryText-sc-1gt7vz8-5 MiXMV'})
    time = div.find('div', {'class': 'RecipeItemstyle__Time-sc-1gt7vz8-6 kKXboO'})

    recipes['title'].append(title.text)
    recipes['subtitle'].append(subtitle.text)
    recipes['img_url'].append(img_url)
    recipes['level'].append(level.text)
    recipes['time'].append(time.text.replace('\n', ''))

# <div class="RecipeListstyle__RecipeList-sc-1s9b4ly-20 eKgfiL">
div_a_class = soup.find('div', {'class':'RecipeListstyle__RecipeList-sc-1s9b4ly-20 eKgfiL'})
a_link_list = div_a_class.find_all('a')


In [121]:
recipes['time']

['30분',
 '25분',
 '30분',
 '10분',
 '20분',
 '20분',
 '10분',
 '30분',
 '20분',
 '20분',
 '30분',
 '20분',
 '40분',
 '1시간 30분',
 '20분',
 '40분',
 '25분',
 '25분',
 '2시간',
 '50분',
 '20분',
 '30분',
 '30분',
 '50분',
 '30분',
 '35분',
 '50분',
 '30분',
 '20분',
 '30분',
 '40분',
 '20분',
 '30분',
 '30분',
 '15분',
 '35분',
 '45분',
 '35분',
 '35분',
 '30분',
 '35분',
 '40분',
 '1시간',
 '40분',
 '20분',
 '25분',
 '30분',
 '20분',
 '1시간',
 '25분',
 '20분',
 '20분',
 '15분',
 '20분',
 '20분',
 '30분',
 '18분',
 '30분',
 '17분',
 '30분',
 '25분',
 '15분',
 '10분',
 '15분',
 '30분',
 '20분',
 '25분',
 '30분',
 '35분',
 '1시간 40분',
 '2시간 30분',
 '30분',
 '3시간 20분',
 '30분',
 '30분',
 '25분',
 '30분',
 '20분',
 '20분',
 '20분',
 '30분',
 '25분',
 '30분',
 '30분',
 '20분',
 '30분',
 '30분',
 '20분',
 '20분',
 '20분',
 '20분',
 '25분',
 '25분',
 '50분',
 '30분',
 '30분',
 '1시간 30분',
 '30분',
 '1시간 20분',
 '40분',
 '40분',
 '40분',
 '30분',
 '25분',
 '20분',
 '20분',
 '30분',
 '30분',
 '20분',
 '1시간 30분',
 '30분',
 '30분',
 '20분',
 '25분',
 '1시간 30분',
 '25분',
 '20분',
 '50분',
 '30분',
 '20분',
 '40분',


In [107]:
for div in div_list:
    print(div)
    print()

<div class="RecipeItemstyle__Layout-sc-1gt7vz8-0 erZvWP">
<img class="RecipeItemstyle__ImageStyle-sc-1gt7vz8-1 duQJWI" src="https://static.wtable.co.kr/image/production/service/recipe/2344/9a1bd1e4-cbf9-4e2c-857a-56c95078bf4c.jpg?size=500x500"/>
<p class="RecipeItemstyle__SubTitle-sc-1gt7vz8-2 LxJcT">LG 광파오븐 레시피</p>
<p class="RecipeItemstyle__Title-sc-1gt7vz8-3 hpYiJK">고추전</p>
<div class="RecipeItemstyle__Summary-sc-1gt7vz8-4 XtlHh">
<div class="RecipeItemstyle__Time-sc-1gt7vz8-6 RecipeItemstyle__Level-sc-1gt7vz8-7 gvzBms">
<svg fill="none" height="12" viewbox="0 0 12 12" width="12" xmlns="http://www.w3.org/2000/svg">
<path d="M10.1.996v10m-4-7.992v8m-4-6v6" stroke="#757575"></path>
</svg>
<span class="RecipeItemstyle__SummaryText-sc-1gt7vz8-5 MiXMV">중급</span>
</div>
<div class="RecipeItemstyle__Time-sc-1gt7vz8-6 kKXboO">
<svg fill="none" height="12" viewbox="0 0 12 12" width="12" xmlns="http://www.w3.org/2000/svg">
<circle cx="6" cy="6" r="5.5" stroke="#757575"></circle>
<path d="M5.5

In [125]:
for a in a_link_list:
    links = [a['href'] for a in a_link_list if 'href' in a.attrs]

# 결과 출력
for link in links:
    root_url = 'https://wtable.co.kr'
    url = root_url + link
    recipes['link'].append(url)
    print(link)

/recipes/SyRXB1s98K1pUgygqB9wwPHD?location=recipe_home
/recipes/UasAyakY1sXVT9QzESDSPz8W?location=recipe_home
/recipes/aqnyvZDijf8Ui8o4GLFnjQGB?location=recipe_home
/recipes/ZpPmCXsRTNydMf2pkqR13HzH?location=recipe_home
/recipes/j7EELtk9r1nfhGShBvrtQR6X?location=recipe_home
/recipes/mke5rs9CEfvmDdHsrxhYEtwn?location=recipe_home
/recipes/1sMUNTAgz4b8XvmPfcq7aQfs?location=recipe_home
/recipes/15JnhcCfSj8uWtsDC8TZ85ED?location=recipe_home
/recipes/4GW6AeEXGSfvzKR5YBd1TyZp?location=recipe_home
/recipes/tJApvajFjwWHn5w5ziwKpNrh?location=recipe_home
/recipes/sSGPDgR7WQWMQ1kiKk3yiG3b?location=recipe_home
/recipes/Hx7Wk3qJVqtQ9R4VevXiJ4X3?location=recipe_home
/recipes/qGNYgFpWiYsir53h8FAUzHPZ?location=recipe_home
/recipes/QzEpZUQgqvrVD8raoiYVrATH?location=recipe_home
/recipes/dG56umvsvRDoH8kMQaYu2zTJ?location=recipe_home
/recipes/tg2DqpEVwGzQMS66bSehzAxQ?location=recipe_home
/recipes/V6Y9nsDkHCW64RaHET3ohCiL?location=recipe_home
/recipes/QDk9nnKsEerfspNHrRtT9Gpp?location=recipe_home
/recipes/8

In [133]:
root_url = 'https://wtable.co.kr'

element_dict = {}

for link in links:
    url = root_url + link
    element_dict[link] = []
    
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    
    ul_class = soup.find('ul', {'class':'igroups'})
    div_ele_list = ul_class.find_all('div', {'class', 'Text__Description02-sc-1qy6bx2-0 fCbbYE'})
    
    for idx, div in enumerate(div_ele_list):
        div_ = div.find_all('div')[0].text
        
        try:
            div_ = div_.split('(')[0]
        except:
            div_ = div_
        element_dict[link].append(div_)
        recipes['element'][idx].extend(div_)
        print(f"{div_}")
    print()

아삭이고추
다진 돼지고기
두부
양파
부침가루
계란 흰자
홍고추
굴소스
청주
다진 마늘
생강가루
참기름
소금
후춧가루
달걀
부침가루
진간장
물
설탕
식초
다진 양파
청고추

황태
밀가루
전분
식용유
간장
올리고당
갈색 설탕
맛술
다진 마늘
다진 파
참기름
통깨

성게
건 미역
다시마 육수
국간장
참기름
멸치액젓
소금

멍게
부추
청양고추
밥
다진 마늘
소금
참기름
깨소금

생 곤드레
식용유
소금
부침가루
물
간장
맛술
식초
잘게 썬 양파
잘게 썬 홍고추
통깨

불린 쌀
물
완두콩

불린 미역
생수
오이
통깨
얼음
식초
설탕
국간장
소금
다진 마늘

부지깽이
식용유
도토리가루
물
간장
식초
잘게 썬 양파
홍고추
통깨

아욱
멸치 다시마 육수
대파
청양고추
된장
고추장
다진 마늘

근대
양파
청양고추
대파
된장
고춧가루
다진 마늘
멸치 다시마 육수

가래떡
양파
파프리카
피망
생표고버섯
잡채용 소고기
만가닥버섯
생목이버섯
다진 잣
통깨
참기름
간장
설탕
참기름
후춧가루
간장
설탕
미림
굴소스

고등어 필렛
오일
소금
후춧가루

도토리가루
물
소금
들기름
진간장
다진 마늘
고춧가루
다진 청양고추
참기름
설탕
통깨
후춧가루

목살
신김치
양파
대파
홍고추
청양고추
멸치다시마육수
된장
김치국물
국간장
맛술
고춧가루
설탕
다진 마늘
후춧가루

배
불고깃감 쇠고기
도라지
대추
간장
맛술
꿀
다진 마늘
후춧가루

표고
오이
당근
숙주
애호박
우엉
목이버섯
소금
식용유
밀가루
물
소금
참기름
간장
맛술
설탕
다진 마늘
참기름
후춧가루
연겨자
간장
식초
설탕
후춧가루

가지
양파
전분
쪽파
식용유
통깨
후춧가루
소금
설탕
물
간장
미림
청주
설탕
생강가루

신김치
중면
오이
깨
고추장
식초
올리고당
간장
설탕
맛술
참기름
후추

열무
얼갈이
쪽파
양파
홍고추
생강
새우젓
굵은 소금
고춧가루
참치액젓
다진 마늘
매실청
소금
밀가루
디포리 다시마 육수

자른 쇠고기 양지
자른 미역
참기름 
국간장
다진 마늘
물 
멸치 액젓 

당근
전분
소금
식용유

신김치
두부
느타리 한줌
양파


In [110]:
element_dict['/recipes/1BPyuRhoNeLAL1gzPhw5XZhy?location=recipe_home']

['닭',
 '숙주',
 '불린 고사리',
 '느타리버섯',
 '양파',
 '대파',
 '무',
 '고추기름',
 '소금',
 '물',
 '통마늘',
 '대파',
 '양파',
 '생강',
 '통후추',
 '국간장',
 '고춧가루',
 '맛술',
 '다진 마늘',
 '다진 파',
 '소금',
 '후춧가루']

In [134]:
recipes['element']

[['아삭이고추',
  '다진 돼지고기',
  '두부',
  '양파',
  '부침가루',
  '계란 흰자',
  '홍고추',
  '굴소스',
  '청주',
  '다진 마늘',
  '생강가루',
  '참기름',
  '소금',
  '후춧가루',
  '달걀',
  '부침가루',
  '진간장',
  '물',
  '설탕',
  '식초',
  '다진 양파',
  '청고추',
  '아',
  '삭',
  '이',
  '고',
  '추',
  '황',
  '태',
  '성',
  '게',
  '멍',
  '게',
  '생',
  ' ',
  '곤',
  '드',
  '레',
  '불',
  '린',
  ' ',
  '쌀',
  '불',
  '린',
  ' ',
  '미',
  '역',
  '부',
  '지',
  '깽',
  '이',
  '아',
  '욱',
  '근',
  '대',
  '가',
  '래',
  '떡',
  '고',
  '등',
  '어',
  ' ',
  '필',
  '렛',
  '도',
  '토',
  '리',
  '가',
  '루',
  '목',
  '살',
  '배',
  '표',
  '고',
  '가',
  '지',
  '신',
  '김',
  '치',
  '열',
  '무',
  '자',
  '른',
  ' ',
  '쇠',
  '고',
  '기',
  ' ',
  '양',
  '지',
  '당',
  '근',
  '신',
  '김',
  '치',
  '표',
  '고',
  '버',
  '섯',
  '냉',
  '이',
  '얼',
  '갈',
  '이',
  '배',
  '추',
  '청',
  '양',
  '고',
  '추',
  '불',
  '린',
  ' ',
  '쌀',
  '비',
  '타',
  '민',
  '불',
  '린',
  ' ',
  '쌀',
  '불',
  '린',
  ' ',
  '쌀',
  '불',
  '린',
  ' ',
  '쌀',
  '불',
  '린',
  ' ',
  '쌀',
  '불',
  '린',
  ' 

In [131]:
recipes['element'] = []
for key,vlaue in enumerate(element_dict.items()):
    recipes['element'].append(vlaue[1])
recipes['element']

[['아삭이고추',
  '다진 돼지고기',
  '두부',
  '양파',
  '부침가루',
  '계란 흰자',
  '홍고추',
  '굴소스',
  '청주',
  '다진 마늘',
  '생강가루',
  '참기름',
  '소금',
  '후춧가루',
  '달걀',
  '부침가루',
  '진간장',
  '물',
  '설탕',
  '식초',
  '다진 양파',
  '청고추'],
 ['황태',
  '밀가루',
  '전분',
  '식용유',
  '간장',
  '올리고당',
  '갈색 설탕',
  '맛술',
  '다진 마늘',
  '다진 파',
  '참기름',
  '통깨'],
 ['성게', '건 미역', '다시마 육수', '국간장', '참기름', '멸치액젓', '소금'],
 ['멍게', '부추', '청양고추', '밥', '다진 마늘', '소금', '참기름', '깨소금'],
 ['생 곤드레',
  '식용유',
  '소금',
  '부침가루',
  '물',
  '간장',
  '맛술',
  '식초',
  '잘게 썬 양파',
  '잘게 썬 홍고추',
  '통깨'],
 ['불린 쌀', '물', '완두콩'],
 ['불린 미역', '생수', '오이', '통깨', '얼음', '식초', '설탕', '국간장', '소금', '다진 마늘'],
 ['부지깽이', '식용유', '도토리가루', '물', '간장', '식초', '잘게 썬 양파', '홍고추', '통깨'],
 ['아욱', '멸치 다시마 육수', '대파', '청양고추', '된장', '고추장', '다진 마늘'],
 ['근대', '양파', '청양고추', '대파', '된장', '고춧가루', '다진 마늘', '멸치 다시마 육수'],
 ['가래떡',
  '양파',
  '파프리카',
  '피망',
  '생표고버섯',
  '잡채용 소고기',
  '만가닥버섯',
  '생목이버섯',
  '다진 잣',
  '통깨',
  '참기름',
  '간장',
  '설탕',
  '참기름',
  '후춧가루',
  '간장',
  '설탕',
  '미림',
  '굴소스'],
 ['고등어

In [112]:
import pandas as pd

In [126]:
print(len(recipes['element']))
print(len(recipes['img_url']))
print(len(recipes['level']))
print(len(recipes['link']))
print(len(recipes['subtitle']))
print(len(recipes['time']))
print(len(recipes['title']))


422
422
422
422
422
422
422


In [160]:
recipes_df = pd.DataFrame(recipes)
recipes_df

,title,subtitle,img_url,level,link,time,element
0,고추전,LG 광파오븐 레시피,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/SyRXB1s98K1pUgygq...,30분,"[아삭이고추, 다진 돼지고기, 두부, 양파, 부침가루, 계란 흰자, 홍고추, 굴소스..."
1,황태갈비,LG 광파오븐 레시피,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/UasAyakY1sXVT9QzE...,25분,"[황태, 밀가루, 전분, 식용유, 간장, 올리고당, 갈색 설탕, 맛술, 다진 마늘,..."
2,성게미역국,귀한 성게가 듬뿍,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/aqnyvZDijf8Ui8o4G...,30분,"[성게, 건 미역, 다시마 육수, 국간장, 참기름, 멸치액젓, 소금, 두, 부, 전..."
3,멍게비빔밥,바다 향 가득,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/ZpPmCXsRTNydMf2pk...,10분,"[멍게, 부추, 청양고추, 밥, 다진 마늘, 소금, 참기름, 깨소금, 양, 파, 식..."
4,곤드레전,비 오는 날엔,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/j7EELtk9r1nfhGShB...,20분,"[생 곤드레, 식용유, 소금, 부침가루, 물, 간장, 맛술, 식초, 잘게 썬 양파,..."
...,...,...,...,...,...,...,...
417,돼지고기장조림,안심살로 만들어 더 부드러운,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/y4wwLB1sGGjdCLosW...,50분,"[돼지고기, 달걀, 물, 대파, 양파, 마늘, 생강, 통후추, 월계수잎, 물, 간장..."
418,약식,동그란 보름달을 닮은,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/71sGWbjPuEFnkAPrQ...,50분,"[찹쌀, 대추, 밤, 잣, 물, 참기름, 대추 씨 끓인 물, 꿀, 간장, 계피가루,..."
419,주꾸미볶음,봄의 영양이 듬뿍 담긴,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/1NKrRoLSiCRZ7Rm1b...,35분,"[주꾸미, 양파, 당근, 대파, 청양고추, 홍고추, 소면, 청주, 소금, 참기름, ..."
420,가지볶음,보라색 채소의 최강자!,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/fZuC5DSY4QtNTr7pr...,20분,"[가지, 양파, 청양고추, 홍고추, 대파, 참기름, 통깨, 식용유, 간장, 굴소스,..."


In [161]:
recipes_df['level'].value_counts()

level
초급    271
중급    146
고급      5
Name: count, dtype: int64

In [162]:
recipes_df['time'].value_counts()

time
30분        120
20분         72
40분         55
25분         36
35분         33
15분         21
50분         19
1시간         13
10분         10
1시간 30분     10
45분          8
1시간 20분      5
2시간          3
1시간 10분      3
1시간 40분      2
55분          1
1시간 15분      1
1시간 45분      1
2시간 10분      1
6시간          1
17분          1
05분          1
39분          1
3시간 20분      1
2시간 30분      1
18분          1
31분          1
Name: count, dtype: int64

In [163]:
recipes_df['level_code'] = 0

easy_idx = recipes_df[recipes_df['level'] == '초급'].index
normal_idx = recipes_df[recipes_df['level'] == '중급'].index
hard_idx = recipes_df[recipes_df['level'] == '고급'].index

recipes_df.loc[easy_idx, 'level_code'] = 0
recipes_df.loc[normal_idx, 'level_code'] = 1
recipes_df.loc[hard_idx, 'level_code'] = 1

recipes_df

,title,subtitle,img_url,level,link,time,element,level_code
0,고추전,LG 광파오븐 레시피,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/SyRXB1s98K1pUgygq...,30분,"[아삭이고추, 다진 돼지고기, 두부, 양파, 부침가루, 계란 흰자, 홍고추, 굴소스...",1
1,황태갈비,LG 광파오븐 레시피,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/UasAyakY1sXVT9QzE...,25분,"[황태, 밀가루, 전분, 식용유, 간장, 올리고당, 갈색 설탕, 맛술, 다진 마늘,...",0
2,성게미역국,귀한 성게가 듬뿍,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/aqnyvZDijf8Ui8o4G...,30분,"[성게, 건 미역, 다시마 육수, 국간장, 참기름, 멸치액젓, 소금, 두, 부, 전...",0
3,멍게비빔밥,바다 향 가득,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/ZpPmCXsRTNydMf2pk...,10분,"[멍게, 부추, 청양고추, 밥, 다진 마늘, 소금, 참기름, 깨소금, 양, 파, 식...",0
4,곤드레전,비 오는 날엔,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/j7EELtk9r1nfhGShB...,20분,"[생 곤드레, 식용유, 소금, 부침가루, 물, 간장, 맛술, 식초, 잘게 썬 양파,...",0
...,...,...,...,...,...,...,...,...
417,돼지고기장조림,안심살로 만들어 더 부드러운,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/y4wwLB1sGGjdCLosW...,50분,"[돼지고기, 달걀, 물, 대파, 양파, 마늘, 생강, 통후추, 월계수잎, 물, 간장...",1
418,약식,동그란 보름달을 닮은,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/71sGWbjPuEFnkAPrQ...,50분,"[찹쌀, 대추, 밤, 잣, 물, 참기름, 대추 씨 끓인 물, 꿀, 간장, 계피가루,...",1
419,주꾸미볶음,봄의 영양이 듬뿍 담긴,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/1NKrRoLSiCRZ7Rm1b...,35분,"[주꾸미, 양파, 당근, 대파, 청양고추, 홍고추, 소면, 청주, 소금, 참기름, ...",1
420,가지볶음,보라색 채소의 최강자!,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/fZuC5DSY4QtNTr7pr...,20분,"[가지, 양파, 청양고추, 홍고추, 대파, 참기름, 통깨, 식용유, 간장, 굴소스,...",0


In [164]:
features =[]
for i in range(len(recipes_df.index)):
    title = str(recipes_df.iloc[i, 1])
    title = re.sub('[^ㄱ-ㅎ가-힣]+', ' ', title)
    subtitle = recipes_df.iloc[i, 0]
    element = str(recipes_df.iloc[i, 6])
    element = re.sub('[^ㄱ-ㅎ가-힣]+', ' ', element)
    
    feature = title + ' ' + subtitle + ' ' + element
    features.append(feature)
    print(feature)
    

 광파오븐 레시피 고추전  아삭이고추 다진 돼지고기 두부 양파 부침가루 계란 흰자 홍고추 굴소스 청주 다진 마늘 생강가루 참기름 소금 후춧가루 달걀 부침가루 진간장 물 설탕 식초 다진 양파 청고추 아 삭 이 고 추 황 태 성 게 멍 게 생 곤 드 레 불 린 쌀 불 린 미 역 부 지 깽 이 아 욱 근 대 가 래 떡 고 등 어 필 렛 도 토 리 가 루 목 살 배 표 고 가 지 신 김 치 열 무 자 른 쇠 고 기 양 지 당 근 신 김 치 표 고 버 섯 냉 이 얼 갈 이 배 추 청 양 고 추 불 린 쌀 비 타 민 불 린 쌀 불 린 쌀 불 린 쌀 불 린 쌀 불 린 쌀 두 부 불 린 쌀 돼 지 고 기 굴 소 고 기 북 어 채 대 패 삼 겹 살 소 고 기 대 패 삼 겹 살 닭 홍 합 감 자 새 우 프 랑 크 소 시 지 두 부 참 가 자 미 무 우 섬 초 조 선 호 박 데 친 쑥 미 더 덕 꼬 막 검 은 콩 통 마 늘 고 구 마 시 금 치 불 린 쌀 닭 가 슴 살 현 미 밥 김 달 걀 배 추 김 치 가 지 묵 은 지 대 하 케 일 소 고 기 양 지 김 밥 용 김 자 숙 홍 합 살 갈 비 새 꼬 막 명 란 젓 다 진 소 고 기 두 부 청 양 고 추 두 부 두 부 감 자 감 자 건 당 면 멸 치 다 시 육 수 떡 국 용 떡 시 판 된 장 떡 국 용 떡 봄 동 봄 동 봄 동 브 로 콜 리 굴 가 지 낙 지 차 돌 박 이 깻 잎 소 고 기 갈 비 배 추 돼 지 고 기 현 미 밥 삶 은 시 래 기 나 물 달 걀 양 배 추 양 배 추 스 팸 단 무 지 깻 잎 부 침 두 부 소 갈 비 밥 가 지 순 두 부 무 삼 겹 살 묵 은 지 청 양 고 추 쌀 배 추 콩 나 물 건 미 역 참 치 캔 달 걀 두 부 달 걀 굴 손 질 한 늙 은 호 박 밥 고 사 리 통 삼 겹 살 무 국 수 호 박 손 질 낙 지 단 감 잘 게 썬 배 추 김 치 쪽 파 굴 두 부 육 전 배 추 소 갈 비 애 호 박 모 둠 건 해 초 배 추 모 둠 건 해 초 삼 겹 살 또 는 오 겹 살 쇠 고 기 쌀 가 지 손 질 된 한 

In [165]:
recipes_df['feature'] = features
recipes_df

,title,subtitle,img_url,level,link,time,element,level_code,feature
0,고추전,LG 광파오븐 레시피,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/SyRXB1s98K1pUgygq...,30분,"[아삭이고추, 다진 돼지고기, 두부, 양파, 부침가루, 계란 흰자, 홍고추, 굴소스...",1,광파오븐 레시피 고추전 아삭이고추 다진 돼지고기 두부 양파 부침가루 계란 흰자 ...
1,황태갈비,LG 광파오븐 레시피,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/UasAyakY1sXVT9QzE...,25분,"[황태, 밀가루, 전분, 식용유, 간장, 올리고당, 갈색 설탕, 맛술, 다진 마늘,...",0,광파오븐 레시피 황태갈비 황태 밀가루 전분 식용유 간장 올리고당 갈색 설탕 맛술...
2,성게미역국,귀한 성게가 듬뿍,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/aqnyvZDijf8Ui8o4G...,30분,"[성게, 건 미역, 다시마 육수, 국간장, 참기름, 멸치액젓, 소금, 두, 부, 전...",0,귀한 성게가 듬뿍 성게미역국 성게 건 미역 다시마 육수 국간장 참기름 멸치액젓 소...
3,멍게비빔밥,바다 향 가득,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/ZpPmCXsRTNydMf2pk...,10분,"[멍게, 부추, 청양고추, 밥, 다진 마늘, 소금, 참기름, 깨소금, 양, 파, 식...",0,바다 향 가득 멍게비빔밥 멍게 부추 청양고추 밥 다진 마늘 소금 참기름 깨소금 양...
4,곤드레전,비 오는 날엔,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/j7EELtk9r1nfhGShB...,20분,"[생 곤드레, 식용유, 소금, 부침가루, 물, 간장, 맛술, 식초, 잘게 썬 양파,...",0,비 오는 날엔 곤드레전 생 곤드레 식용유 소금 부침가루 물 간장 맛술 식초 잘게 ...
...,...,...,...,...,...,...,...,...,...
417,돼지고기장조림,안심살로 만들어 더 부드러운,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/y4wwLB1sGGjdCLosW...,50분,"[돼지고기, 달걀, 물, 대파, 양파, 마늘, 생강, 통후추, 월계수잎, 물, 간장...",1,안심살로 만들어 더 부드러운 돼지고기장조림 돼지고기 달걀 물 대파 양파 마늘 생강...
418,약식,동그란 보름달을 닮은,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/71sGWbjPuEFnkAPrQ...,50분,"[찹쌀, 대추, 밤, 잣, 물, 참기름, 대추 씨 끓인 물, 꿀, 간장, 계피가루,...",1,동그란 보름달을 닮은 약식 찹쌀 대추 밤 잣 물 참기름 대추 씨 끓인 물 꿀 간장...
419,주꾸미볶음,봄의 영양이 듬뿍 담긴,https://static.wtable.co.kr/image/production/s...,중급,https://wtable.co.kr/recipes/1NKrRoLSiCRZ7Rm1b...,35분,"[주꾸미, 양파, 당근, 대파, 청양고추, 홍고추, 소면, 청주, 소금, 참기름, ...",1,봄의 영양이 듬뿍 담긴 주꾸미볶음 주꾸미 양파 당근 대파 청양고추 홍고추 소면 청...
420,가지볶음,보라색 채소의 최강자!,https://static.wtable.co.kr/image/production/s...,초급,https://wtable.co.kr/recipes/fZuC5DSY4QtNTr7pr...,20분,"[가지, 양파, 청양고추, 홍고추, 대파, 참기름, 통깨, 식용유, 간장, 굴소스,...",0,보라색 채소의 최강자 가지볶음 가지 양파 청양고추 홍고추 대파 참기름 통깨 식용...


In [166]:
recipes_df['id'] = recipes_df.index+1
recipes_df=recipes_df[['id', 'subtitle', 'title', 'img_url', 'level', 'level_code', 'link', 'time', 'element', 'feature']]
recipes_df

,id,subtitle,title,img_url,level,level_code,link,time,element,feature
0,1,LG 광파오븐 레시피,고추전,https://static.wtable.co.kr/image/production/s...,중급,1,https://wtable.co.kr/recipes/SyRXB1s98K1pUgygq...,30분,"[아삭이고추, 다진 돼지고기, 두부, 양파, 부침가루, 계란 흰자, 홍고추, 굴소스...",광파오븐 레시피 고추전 아삭이고추 다진 돼지고기 두부 양파 부침가루 계란 흰자 ...
1,2,LG 광파오븐 레시피,황태갈비,https://static.wtable.co.kr/image/production/s...,초급,0,https://wtable.co.kr/recipes/UasAyakY1sXVT9QzE...,25분,"[황태, 밀가루, 전분, 식용유, 간장, 올리고당, 갈색 설탕, 맛술, 다진 마늘,...",광파오븐 레시피 황태갈비 황태 밀가루 전분 식용유 간장 올리고당 갈색 설탕 맛술...
2,3,귀한 성게가 듬뿍,성게미역국,https://static.wtable.co.kr/image/production/s...,초급,0,https://wtable.co.kr/recipes/aqnyvZDijf8Ui8o4G...,30분,"[성게, 건 미역, 다시마 육수, 국간장, 참기름, 멸치액젓, 소금, 두, 부, 전...",귀한 성게가 듬뿍 성게미역국 성게 건 미역 다시마 육수 국간장 참기름 멸치액젓 소...
3,4,바다 향 가득,멍게비빔밥,https://static.wtable.co.kr/image/production/s...,초급,0,https://wtable.co.kr/recipes/ZpPmCXsRTNydMf2pk...,10분,"[멍게, 부추, 청양고추, 밥, 다진 마늘, 소금, 참기름, 깨소금, 양, 파, 식...",바다 향 가득 멍게비빔밥 멍게 부추 청양고추 밥 다진 마늘 소금 참기름 깨소금 양...
4,5,비 오는 날엔,곤드레전,https://static.wtable.co.kr/image/production/s...,초급,0,https://wtable.co.kr/recipes/j7EELtk9r1nfhGShB...,20분,"[생 곤드레, 식용유, 소금, 부침가루, 물, 간장, 맛술, 식초, 잘게 썬 양파,...",비 오는 날엔 곤드레전 생 곤드레 식용유 소금 부침가루 물 간장 맛술 식초 잘게 ...
...,...,...,...,...,...,...,...,...,...,...
417,418,안심살로 만들어 더 부드러운,돼지고기장조림,https://static.wtable.co.kr/image/production/s...,중급,1,https://wtable.co.kr/recipes/y4wwLB1sGGjdCLosW...,50분,"[돼지고기, 달걀, 물, 대파, 양파, 마늘, 생강, 통후추, 월계수잎, 물, 간장...",안심살로 만들어 더 부드러운 돼지고기장조림 돼지고기 달걀 물 대파 양파 마늘 생강...
418,419,동그란 보름달을 닮은,약식,https://static.wtable.co.kr/image/production/s...,중급,1,https://wtable.co.kr/recipes/71sGWbjPuEFnkAPrQ...,50분,"[찹쌀, 대추, 밤, 잣, 물, 참기름, 대추 씨 끓인 물, 꿀, 간장, 계피가루,...",동그란 보름달을 닮은 약식 찹쌀 대추 밤 잣 물 참기름 대추 씨 끓인 물 꿀 간장...
419,420,봄의 영양이 듬뿍 담긴,주꾸미볶음,https://static.wtable.co.kr/image/production/s...,중급,1,https://wtable.co.kr/recipes/1NKrRoLSiCRZ7Rm1b...,35분,"[주꾸미, 양파, 당근, 대파, 청양고추, 홍고추, 소면, 청주, 소금, 참기름, ...",봄의 영양이 듬뿍 담긴 주꾸미볶음 주꾸미 양파 당근 대파 청양고추 홍고추 소면 청...
420,421,보라색 채소의 최강자!,가지볶음,https://static.wtable.co.kr/image/production/s...,초급,0,https://wtable.co.kr/recipes/fZuC5DSY4QtNTr7pr...,20분,"[가지, 양파, 청양고추, 홍고추, 대파, 참기름, 통깨, 식용유, 간장, 굴소스,...",보라색 채소의 최강자 가지볶음 가지 양파 청양고추 홍고추 대파 참기름 통깨 식용...


In [167]:
recipes_df.to_csv('./korean_food.csv', encoding='utf-8', index=False)

In [168]:
recipes_df['level_code'].value_counts()

level_code
0    271
1    151
Name: count, dtype: int64